In [24]:
#coding=utf-8

import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import json
import time

# 这里也可以设置IP池
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}       # 这里要填写user-agent
#url = 'http://reportapi.eastmoney.com/report/list?cb=datatable3151164&industryCode=*&pageSize=50&industry=*&rating=*&ratingChange=*&beginTime=2020-03-08&endTime=2022-03-08&pageNo={}&fields=&qType=1&orgCode=&rcode=&_=1646710798362'

url = 'https://reportapi.eastmoney.com/report/jg?cb=datatable3394864&pageSize=50&beginTime=2020-11-12&endTime=2022-11-12&pageNo={}&fields=&qType=3&orgCode=&author=&p=2&pageNum=2&pageNumber=2&_=1668238775186'

def getHtml(pageNo):

    response = requests.get(url.format(pageNo), headers = header)
    html = response.content.decode("utf-8")

    return html



def parse_data(data, News):
        
    for i in data['data']:
        # 作者信息
        researcher = i['researcher']
        orgSName = i['orgSName']
        orgCode = i['orgCode']
        # 研报内容信息
        publishDate = i['publishDate']
        title = i['title']
        url = 'https://data.eastmoney.com/report/zw_macresearch.jshtml?encodeUrl=' + i['encodeUrl']
        Innerhtml = requests.get(url)
        Innerhtml.encoding='utf-8'
        soup = BeautifulSoup(Innerhtml.text,'html.parser')
        page = soup.select('#ctx-content > p')
        content = []
        for para in page:
            if para.text != None:
                content.append(para.text.strip())   
        content_info = ''.join(content)
        
        News.append({'researcher' : researcher,
                         'orgSName' : orgSName,
                         'orgCode' : orgCode,
                         'publishDate' : publishDate,
                         'title' : title,
                         'content' : content_info,
                         'url' : url                    
                        })
    return News



def get_industry_research_paper(start_page=140, end_page=141):
    # 获取一定页数范围内的研报
    News = []
    for page_num in tqdm(np.arange(start_page,end_page)):
        items = getHtml(page_num)
        #print(items)
        items = items.replace('datatable3394864(', '')[:-1]
        data = json.loads(items)
        News = parse_data(data, News)
        #print(data)
        time.sleep(300)

    # 写入    
    with open('industry_research_paper3.json', 'w', encoding='utf-8') as f:
        json.dump(News, f, indent = 1, ensure_ascii = False)

In [25]:
get_industry_research_paper()

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [06:03<00:00, 363.08s/it]
